In [1]:
from datetime import datetime
from sklearn.impute import KNNImputer
import numpy as np
import pandas as pd
import json

In [2]:
data = pd.read_csv('myFitnessPal_parsed.csv')
data.head()

,user_id,date,sequence,food_ids,total_calories,total_carbs,total_fat,total_protein,total_sodium,total_sugar,goal_calories,goal_carbs,goal_fat,goal_protein,goal_sodium,goal_sugar
0,1,2014-09-15,1,"[1, 2, 3, 4, 4]",2430,96,37.0,50.0,855.0,63.0,1572.0,196.0,52.0,79.0,2300.0,59.0
1,1,2014-09-16,1,"[5, 1, 2, 3, 6, 7]",1862,158,54.0,114.0,2215.0,100.0,1832.0,229.0,61.0,92.0,2300.0,69.0
2,1,2014-09-17,1,"[1, 2, 3, 6, 8, 9, 10]",2251,187,60.0,98.0,1765.0,105.0,1685.0,210.0,56.0,85.0,2300.0,63.0
3,1,2014-09-18,1,"[1, 6, 2, 3, 11, 12]",2001,113,81.0,202.0,1101.0,71.0,1597.0,199.0,53.0,80.0,2300.0,60.0
4,1,2014-09-19,1,"[1, 7, 13, 12, 2, 3, 12, 12]",2158,180,89.0,115.0,1998.0,84.0,1589.0,198.0,53.0,80.0,2300.0,60.0


In [3]:
with open('foods.json') as json_file:
     foods = json.load(json_file)

FileNotFoundError: [Errno 2] No such file or directory: 'foods.json'

In [4]:
# # split date and create separated columns
# data["year"] = data["date"].apply(lambda x: int(x[:4]))
# data["month"] = data["date"].apply(lambda x: int(x[5:7]))
# data["day"] = data["date"].apply(lambda x: int(x[-2:]))
# data = data.drop(columns = 'date')

In [3]:
# create new column based on foods quantity
data['foods_len'] = data["food_ids"].apply(lambda x: len(x[1:-1].split(',')))

In [4]:
# create new columns based on goal and total nutritient
data["calories_diff"] = data["goal_calories"]-data["total_calories"]
data["carbs_diff"] = data["goal_carbs"]-data["total_carbs"]
data["fat_diff"] = data["goal_fat"]-data["total_fat"]
data["protein_diff"] = data["goal_protein"]-data["total_protein"]
data["sodium_diff"] = data["goal_sodium"]-data["total_sodium"]
data["sugar_diff"] = data["goal_sugar"]-data["total_sugar"]

In [5]:
#In this cell we drop rows which have null values in more than 4 columns

nullColumns= data.loc[data.isnull().sum(axis=1)>=5]['user_id'].unique()
nullColumns=pd.DataFrame(nullColumns,columns={'user_id'})

cond = data['user_id'].isin(nullColumns['user_id']) 
data.drop(data[cond].index, inplace = True)

In [6]:
# There are no null values for goal_calories after drop action
data[np.isnan(data['goal_calories'])]['user_id'].value_counts()

Series([], Name: user_id, dtype: int64)

In [7]:
# seems like there are people who have set goal_calories to zero and it should also be the mistake
# so lets drop those ones too

zeros= data[data['goal_calories']==0]['user_id'].unique()
zeros=pd.DataFrame(zeros,columns={'user_id'})

cond = data['user_id'].isin(zeros['user_id']) 
data.drop(data[cond].index, inplace = True)

In [8]:
# add new feature which shows how many days are logged by each applicants
user_logged_freq = data["user_id"].value_counts()
user_logged_df = pd.DataFrame(data["user_id"].unique(),columns = ['user_id'])
user_logged_df["logged_frequency"] = user_logged_df["user_id"].apply(lambda _id: user_logged_freq[_id])

In [9]:
# this function counts whole days between start and end date and calculates missed days for new feature
def days_missed(d1, d2,loggedDays):
    d1 = datetime.strptime(str(d1), "%Y-%m-%d")
    d2 = datetime.strptime(str(d2), "%Y-%m-%d")
    return abs(abs((d2 - d1).days)-loggedDays)

In [10]:
# this function calls days_missed regarding last and first records
def get_missed_days(df,userID,logged_frequency):
    tail = df[df["user_id"]==userID].tail(1)['date'].values[0]
    head = df[df["user_id"]==userID].head(1)['date'].values[0]
    return days_missed(tail,head,logged_frequency)   

In [11]:
# add new feature based on how many days are missed for each user
user_logged_df['days_missed'] = user_logged_df[['user_id','logged_frequency']].apply(lambda x: get_missed_days(data,x.user_id,x.logged_frequency),axis=1)

In [12]:
# There is known that for healthy eating, daily carbs should be between 45-65% out of callories, fats between 10-35% and proteins 20-35%
# So this method checks if persons daily norm is healthy distributed

def healthyDistributed(carbs,fat,protein):
    sum = fat*9+ carbs*4 + protein*4 # convert to calories (1g fat = 9 calories and etc..) and sum
    return (0.45 < (carbs*4 /(sum+0.00001)<0.65)) & (0.10 < (fat*9 /(sum+0.00001))<0.35) & (0.20 < (protein*4 /(sum+0.00001))<0.35)

In [13]:
#new column in data based on healthydistributed method to determine user's behavior
data['healtyDistrib']=data[['total_carbs','total_fat','total_protein']].apply(lambda x: healthyDistributed(x.total_carbs,x.total_fat,x.total_protein),axis=1)

In [14]:
data=data.drop(columns=['date','food_ids','sequence'])

In [15]:
# we need to have the same number of rows for each user so
# this function checks if number of rows for each user are greater than num_rows
# if it's greater, then this function returns the lasts rows 
# if less, then this fills them with -1

def row_padding(x,num_row):
    # get last row because I need same format and same id, others columns replaced by -1
    last_row = x.iloc[-1] 
    last_row[1:] = [-1]*len(last_row[1:])
    
    if np.shape(x)[0] < num_row:
        new_x = pd.DataFrame(x)
        for i in range(np.shape(x)[0],num_row):
            new_x = pd.DataFrame(new_x.append(last_row))
        return new_x
    else:
        return x.tail(num_row)

In [16]:
# this function flattens all rows for each user which we padded already 
# so creates one vector because we need one input for each user
def flatten_rows(x,cols):
    for i in range(1,x.shape[0]):
        for j in range(1,len(cols)):
            temp_row = x.iloc[i]
            x[cols[j]+"_"+str(i)] = temp_row[j]
    return x.head(1)

In [17]:
data = data.groupby('user_id').apply(row_padding,5).reset_index(drop=True)

In [18]:
cols = data.columns
data = data.groupby('user_id').apply(flatten_rows,cols).reset_index(drop=True)

In [19]:
data

,user_id,total_calories,total_carbs,total_fat,total_protein,total_sodium,total_sugar,goal_calories,goal_carbs,goal_fat,...,goal_sodium_4,goal_sugar_4,foods_len_4,calories_diff_4,carbs_diff_4,fat_diff_4,protein_diff_4,sodium_diff_4,sugar_diff_4,healtyDistrib_4
0,1,4122,464,196.0,168.0,5787.0,125.0,4578.0,572.0,153.0,...,2300.0,87.0,6.0,1121.0,176.0,34.0,18.0,1162.0,26.0,1.0
1,2,1548,153,52.0,44.0,1752.0,100.0,1320.0,165.0,44.0,...,2300.0,50.0,2.0,1264.0,154.0,43.0,59.0,2193.0,40.0,1.0
2,3,1468,158,41.0,55.0,1588.0,15.0,1486.0,185.0,49.0,...,2300.0,25.0,6.0,2.0,-22.0,10.0,26.0,593.0,14.0,0.0
3,4,645,63,39.0,27.0,1454.0,192.0,1450.0,181.0,73.0,...,2300.0,3500.0,8.0,4.0,-24.0,14.0,4.0,1182.0,1440.0,0.0
4,5,3754,196,33.0,124.0,2569.0,90.0,3399.0,212.0,46.0,...,2300.0,90.0,14.0,155.0,45.0,12.0,56.0,-844.0,-32.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9367,9893,1140,47,49.0,13.0,2000.0,8.0,1932.0,97.0,242.0,...,2300.0,73.0,1.0,1122.0,28.0,244.0,62.0,2140.0,73.0,0.0
9368,9894,1873,237,75.0,64.0,50.0,29.0,1660.0,208.0,55.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
9369,9895,1755,221,54.0,78.0,8.0,85.0,2024.0,202.0,56.0,...,44.0,44.0,19.0,-132.0,-21.0,-46.0,80.0,27.0,-20.0,0.0
9370,9896,338,20,30.0,11.0,8.0,18.0,1290.0,43.0,161.0,...,25.0,48.0,5.0,734.0,21.0,139.0,-8.0,14.0,36.0,0.0


In [45]:
# merge two new features with main data
data = pd.merge(data, user_logged_df, on=['user_id'])

In [20]:
null_value_frequency = data.isna().sum()
null_value_frequency[null_value_frequency!=0] 

goal_sodium      677
goal_sugar       682
sodium_diff      677
sugar_diff       682
goal_sodium_1    642
goal_sugar_1     645
sodium_diff_1    642
sugar_diff_1     645
goal_sodium_2    628
goal_sugar_2     632
sodium_diff_2    628
sugar_diff_2     632
goal_sodium_3    608
goal_sugar_3     610
sodium_diff_3    608
sugar_diff_3     610
goal_sodium_4    599
goal_sugar_4     603
sodium_diff_4    599
sugar_diff_4     603
dtype: int64

In [26]:
# handle missing data with KNNImputer
imputer = KNNImputer()
data[data.columns] = round(imputer.fit_transform(data))
data[data.columns] = round(imputer.fit_transform(data))

In [65]:
null_value_frequency = data.isna().sum()
null_value_frequency[null_value_frequency!=0] 

Series([], dtype: int64)

In [51]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9372 entries, 0 to 9371
Columns: 116 entries, user_id to healtyDistrib_4
dtypes: float64(80), int64(26), object(10)
memory usage: 8.3+ MB


In [24]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [25]:
data.describe()

,user_id,total_calories,total_carbs,total_fat,total_protein,total_sodium,total_sugar,goal_calories,goal_carbs,goal_fat,goal_protein,goal_sodium,goal_sugar,foods_len,calories_diff,carbs_diff,fat_diff,protein_diff,sodium_diff,sugar_diff,healtyDistrib,total_calories_1,total_carbs_1,total_fat_1,total_protein_1,total_sodium_1,total_sugar_1,goal_calories_1,goal_carbs_1,goal_fat_1,goal_protein_1,goal_sodium_1,goal_sugar_1,foods_len_1,calories_diff_1,carbs_diff_1,fat_diff_1,protein_diff_1,sodium_diff_1,sugar_diff_1,healtyDistrib_1,total_calories_2,total_carbs_2,total_fat_2,total_protein_2,total_sodium_2,total_sugar_2,goal_calories_2,goal_carbs_2,goal_fat_2,goal_protein_2,goal_sodium_2,goal_sugar_2,foods_len_2,calories_diff_2,carbs_diff_2,fat_diff_2,protein_diff_2,sodium_diff_2,sugar_diff_2,healtyDistrib_2,total_calories_3,total_carbs_3,total_fat_3,total_protein_3,total_sodium_3,total_sugar_3,goal_calories_3,goal_carbs_3,goal_fat_3,goal_protein_3,goal_sodium_3,goal_sugar_3,foods_len_3,calories_diff_3,carbs_diff_3,fat_diff_3,protein_diff_3,sodium_diff_3,sugar_diff_3,healtyDistrib_3,total_calories_4,total_carbs_4,total_fat_4,total_protein_4,total_sodium_4,total_sugar_4,goal_calories_4,goal_carbs_4,goal_fat_4,goal_protein_4,goal_sodium_4,goal_sugar_4,foods_len_4,calories_diff_4,carbs_diff_4,fat_diff_4,protein_diff_4,sodium_diff_4,sugar_diff_4,healtyDistrib_4
count,9372.000000,9372.000000,9372.000000,9372.000000,9372.000000,9372.000000,9372.000000,9372.000000,9372.000000,9372.000000,9372.000000,8695.000000,8690.000000,9372.000000,9372.000000,9372.000000,9372.000000,9372.000000,8695.000000,8690.000000,9372.000000,9372.000000,9372.000000,9372.000000,9372.000000,9372.000000,9372.000000,9372.000000,9372.000000,9372.000000,9372.000000,8730.000000,8727.000000,9372.000000,9372.000000,9372.000000,9372.000000,9372.000000,8730.000000,8727.000000,9372.000000,9372.000000,9372.000000,9372.000000,9372.000000,9372.000000,9372.000000,9372.000000,9372.000000,9372.000000,9372.000000,8744.000000,8740.000000,9372.000000,9372.000000,9372.000000,9372.000000,9372.000000,8744.000000,8740.000000,9372.000000,9372.000000,9372.000000,9372.000000,9372.000000,9372.000000,9372.000000,9372.000000,9372.000000,9372.000000,9372.000000,8764.000000,8762.000000,9372.000000,9372.000000,9372.000000,9372.000000,9372.000000,8764.000000,8762.000000,9372.000000,9372.000000,9372.000000,9372.000000,9372.000000,9372.000000,9372.000000,9372.000000,9372.000000,9372.000000,9372.000000,8773.000000,8769.000000,9372.000000,9372.000000,9372.000000,9372.00000,9372.000000,8773.000000,8769.000000,9372.000000
mean,4940.805271,1307.380388,136.989117,64.107875,81.672962,1126.928617,363.751707,1638.448464,190.843257,88.320529,159.536918,1485.905693,374.947066,9.410691,331.068075,53.854140,24.212655,77.863956,412.614146,120.817837,0.189287,1186.796735,125.503094,57.821169,77.059646,965.674349,343.472151,1565.062527,181.512591,84.333440,149.750000,1422.304238,356.513922,8.958387,378.223005,55.966709,26.469484,72.647567,412.714662,118.843933,0.143939,1148.569356,121.887431,56.289479,73.532757,934.191101,337.692382,1521.423175,177.631562,82.265258,147.723645,1374.375114,348.207895,8.610969,372.782330,55.672642,25.904289,74.119398,397.399016,114.979748,0.108835,1083.964895,117.471831,53.568609,67.972258,866.290546,319.765685,1477.227700,170.711801,79.842936,142.503735,1340.207325,337.079890,8.204225,393.167520,53.144686,26.179044,74.436193,437.482200,118.743438,0.076504,878.189714,93.108621,44.394473,57.126761,719.181285,278.423175,1422.166773,164.642979,76.799189,138.299509,1313.207227,329.841145,6.796095,543.863743,71.421041,32.29140,81.059432,563.820130,151.514996,0.043854
std,2861.574643,5504.800538,512.158720,249.489011,235.625471,9993.263384,788.847121,648.843278,206.542568,233.227354,365.806683,1111.631917,834.372498,5.933921,5535.366504,517.043113,223.285919,334.526569,1115.095117,556.756391,0.391758,701.930440,182.539266,176.560561,212.813225,1284.795684,774.737691,707.001935,203.792222,228.2870

# update dataframe

In [1]:
data.to_csv(r'myFitnessPal_parsed.csv',index = False)

NameError: name 'data' is not defined